# iLINCS API R Notebook

## Prerequisites

In [ ]:
library(IRdisplay)
library(tinytex)
library(httr)
library(jsonlite)
library(htmltools)

if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("Biobase")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


Bioconductor version 3.15 (BiocManager 1.30.18), R 4.2.0 (2022-04-22)

Installing package(s) 'BiocVersion', 'Biobase'

also installing the dependency ‘BiocGenerics’


Old packages: 'openssl', 'roxygen2', 'tinytex', 'usethis', 'xfun'



In [ ]:
library("readxl")
library(dplyr)

my_data <- read_excel("5_compounds_for_drug_combination.xlsx", sheet="Sheet1")
mydata_sub <- my_data[my_data$Concordance < -0.3 & my_data$pValue < 0.01, ]
mydata_sub_unique = distinct(mydata_sub, Perturbagen, Concentration, .keep_all= TRUE)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
#item_list = list("LINCSCP_32475", "LINCSCP_132961", "LINCSCP_38805", "LINCSCP_35665", "LINCSCP_133061", "LINCSCP_136993")
item_list <- as.list(mydata_sub_unique$SignatureId)
for (item in item_list){
  ilincs_signatureId <- item
  req <- POST("http://www.ilincs.org/api/ilincsR/downloadSignature", body = list(sigID = paste(ilincs_signatureId), display = FALSE), encode = "json")
  ilincs_sessionId<-unlist(httr::content(req))

  fileUrl<-paste("http://www.ilincs.org/tmp/",ilincs_sessionId,".xls",sep="")
  signatureData<-read.table(fileUrl,sep="\t",header=T,stringsAsFactors = F)
  write.csv(signatureData,paste(ilincs_signatureId,".csv"), row.names = FALSE)
}

In [ ]:
library(zip)
Zip_Files <- list.files(path = getwd(), pattern = ".csv$")
# Zip the files and place the zipped file in working directory
zip::zip(zipfile = "LINSCP_for_drug_combinations.zip", files = Zip_Files)